In [1]:
#coding: utf-8
import numpy as np
import pandas as pd
import re
import jieba
import os
import time
import jieba  
import numpy as np
from progressbar import *
start1 =time.clock()

C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()


In [2]:
def obatin_news_from_csv(path_csv):
    old_text=pd.read_csv(path_csv)
    drop_index_na=[]
    for i in range(old_text.shape[0]):
        j=old_text['微博正文'][i]
        if ("抱歉" in j or len(j)<40):
            drop_index_na.append(i)
    text=old_text.drop(drop_index_na)
    text=text.reset_index(drop=True)
    return text

import re
def find_title(text):
    text["标题"]=[-99 for i in range(text.shape[0])]
    title_p=re.compile(r".*\【.*\】")
    progress = ProgressBar()
    for i in progress(range(len(text["微博正文"]))):
        title_=re.match(title_p,text["微博正文"][i])
        if title_:
            text["标题"][i]=title_[0]
        text["微博正文"][i]=re.sub(".*\【.*\】","",text["微博正文"][i])
        time.sleep(0.1)
    notitle_index=[]
    for i in range(text.shape[0]):
        if text["标题"][i]==-99:
            notitle_index.append(i)
    text=text.drop(notitle_index)
    text=text.reset_index(drop=True)
    return text

In [3]:
#载入词典和停用词
jieba.load_userdict(r".\环球时报\自定义词典.txt")
stopwords = [line.strip() for line in open(r".\环球时报\停用词库.txt",encoding='UTF-8').readlines()]
def clean_text_content(text_content,cut_all=False):
    text_content = re.sub(r"。", ".", text_content)
    text_content = re.sub(r"，", ",", text_content)
    text_content = re.sub(r"“", "'", text_content)
    text_content = re.sub(r"”", "'", text_content)
    text_content = re.sub(r"…", ".", text_content)
    text_content = re.sub(r"@", "@", text_content)
    text_content = re.sub(r" ", " ", text_content)
    text_content = re.sub(r"！", "!", text_content)
    text_content = re.sub(r"？", "?", text_content)
    text_content = re.sub(r"：", ":", text_content)
    text_content = re.sub(r"）", ")", text_content)
    text_content = re.sub(r"（", "(", text_content)
    text_content = re.sub(r"(\d+年)*(\d+月)*(\d+[日])", "", text_content) #日期
    text_content = re.sub(r"\d+[年月日天号人名时例名省市区县院]", "", text_content) 
    text_content = re.sub(r"[第]*[零一二三四五六七八九百千万]+[年月日天号人名时例名省市区县院名例周月年]*", "", text_content) 
    text_content = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text_content) #时间
    text_content = re.sub(r"^[-+]?[0-9]+(\.)?[0-9]*$", "", text_content) #数字
    text_content = re.sub(r"@\S*\s", " ", text_content) #@小央视频
    text_content = re.sub(r"\[.+\]", "", text_content) #[组图共2张]和[加油]
    text_content = re.sub(r"\(.*\)", "", text_content) #（环球网）
    
    text_content = re.sub(r"\W+\w*(视频)\s+", "", text_content) #小央视频的秒拍视频
    text_content = re.sub(r"#", "", text_content) ##
    text_content = re.sub(r"(http|https)(://t.cn/)[a-zA-Z0-9]+", "", text_content) #网址(微博上的连接都是http://t.cn/.....形式)
    text_content = re.sub(r"转发理由:", "", text_content) 
    text_content = re.sub(r"转发内容:", "", text_content) 
    text_content = re.sub(r"原始用户:.*", "", text_content) 
    
    word_list=jieba.lcut(text_content,cut_all=cut_all)
    word_list_len=len(word_list)
    i=0
    while i<word_list_len:
        j=word_list[i]
        if j in stopwords or not('\u4e00'<=j<='\u9fff') or len(j)<2:
            word_list.pop(i)
            word_list_len=word_list_len-1
        elif (j=="中方" or j=="我国"):
            word_list[i]="中国"
        elif j=="美方":
            word_list[i]="美国"
        elif j=="贸易战":
            word_list[i]="贸易" 
        else:
            i=i+1
    if " " in word_list:
        word_list.remove(" ")
    return word_list#text_content 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Public\Documents\Wondershare\CreatorTemp\jieba.cache
Loading model cost 1.929 seconds.
Prefix dict has been built successfully.


In [4]:
import re
a=[
    "./数据集2-五个分类/中国农业新闻网/2106046392.csv",
    "./数据集2-五个分类/新浪教育/1225314032.csv",
    "./数据集2-五个分类/新浪财经/1638782947.csv",
    "./数据集2-五个分类/新华体育/1731169267.csv",
    "./数据集2-五个分类/新华国际/2641686425.csv"
]

text_one_news_list=[]
dict_label={"农业":1,"教育":2,"财经":3,"体育":4,"国际":5}

for i in a:
    
    pattern1=re.compile(r"[\u4e00-\u9fa5]*")
    class_name1=pattern1.findall(i)[2]
    class_name=class_name1[len(class_name1)-2:len(class_name1)]
    if class_name=="闻网":
        class_name="农业"
    text_one_news=obatin_news_from_csv(i)
    text_one_news=find_title(text_one_news)
    text_one_news['分类']=[dict_label[class_name] for j in range(text_one_news.shape[0])]
    
    index_text_class=["微博正文","标题","分类"]
    text_one_news_1=text_one_news[index_text_class]
    
    text_one_news_1=text_one_news_1.drop_duplicates()
    text_one_news_list.append(text_one_news_1)

text_one_news_list2=[]
text_one_news_list2.append(text_one_news_list[0].loc[len(text_one_news_list[0])-1000:len(text_one_news_list[0])])
text_one_news_list2.append(text_one_news_list[1].loc[len(text_one_news_list[1])-1000:len(text_one_news_list[1])])
text_one_news_list2.append(text_one_news_list[2].loc[len(text_one_news_list[2])-1000:len(text_one_news_list[2])])
text_one_news_list2.append(text_one_news_list[3].loc[len(text_one_news_list[3])-1000:len(text_one_news_list[3])])
text_one_news_list2.append(text_one_news_list[4].loc[1900:2900])

for i in text_one_news_list:
    print(i.shape[0])

N/A% (0 of 4257) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dell-pc\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100% (4257 of 4257

4062
3661
4505
5966
3345


In [5]:
text=text_one_news_list2[0]
for i in range(1,5):
    text=text.append(text_one_news_list2[i])
text=text.reset_index(drop=True)
print(text.shape[0])
text.head(5)

4993


,微博正文,标题,分类
0,春分已过，清明将至。农谚讲，清明前后，种瓜点豆。当前全国由南向北全面进入春耕生产大忙季节。...,【农业农村部：当前春耕备耕热潮掀起进展顺利】,1
1,大学生村官制度是我国向农村输血的重要途径。半月谈记者在河北、山东、陕西等地调研发现，各地围绕...,【大学生村官的困惑：困在农村 还是奋斗在农村?】,1
2,从昔日深山的羊肠小道变身今天网友盛赞的“最美赛道”，广西马山县古零镇古零村弄拉屯一条不到4公...,【一个山旮旯的振兴“路”】,1
3,今年82岁的盖钧镒还时常卷起裤腿下到田间地头。身为中国工程院院士、大豆遗传育种学家，他更是将...,#中国种业十大杰出人物#【盖钧镒：一生为了“豆满仓”】,1
4,"又是一年春播时,广大农民和各级农业部门又开始了繁忙的春耕备耕工作。在乡村振兴政策的引导下,今...",【今年春耕有啥新特点】,1


In [6]:


start =time.clock()

len_text=text.shape[0]

text["微博正文(无标题切词后)"]=[-99 for i in range(len_text)]
text["标题(切词后)"]=[-99 for i in range(len_text)]

drop_na=[]

progress = ProgressBar()

for u in progress(range(0,len_text)):
    cleaned_content=clean_text_content(text["微博正文"][u])
    text["微博正文(无标题切词后)"][u]=" ".join(cleaned_content)

    cleaned_title=clean_text_content(text["标题"][u],True)
    text["标题(切词后)"][u]=" ".join(cleaned_title)

    if pd.isnull(text.loc[u]).any():
        drop_na.append(u)
        print(u) 
        
        
    time.sleep(0.1)

text=text.drop(drop_na)
text=text.reset_index(drop=True)   

text["微博正文(有标题切词后)"]=[-99 for i in range(len_text)]
progress = ProgressBar()

for u in progress(range(0,len_text)):
    text["微博正文(有标题切词后)"][u]=text["标题(切词后)"][u]+" "+text["微博正文(无标题切词后)"][u]


end = time.clock()

print('Running time: %s Seconds'%(end-start))
text.head()

C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
N/A% (0 of 4993) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100% (4993 of 4993

Running time: 1057.8040643 Seconds


C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


,微博正文,标题,分类,微博正文(无标题切词后),标题(切词后),微博正文(有标题切词后)
0,春分已过，清明将至。农谚讲，清明前后，种瓜点豆。当前全国由南向北全面进入春耕生产大忙季节。...,【农业农村部：当前春耕备耕热潮掀起进展顺利】,1,春分 已过 清明 农谚 清明 种瓜 点豆 当前 全国 由南向北 全面 春耕生产 大忙 季节 ...,农业 农村 农村部 当前 春耕 备耕 热潮 掀起 进展 顺利,农业 农村 农村部 当前 春耕 备耕 热潮 掀起 进展 顺利 春分 已过 清明 农谚 清明 ...
1,大学生村官制度是我国向农村输血的重要途径。半月谈记者在河北、山东、陕西等地调研发现，各地围绕...,【大学生村官的困惑：困在农村 还是奋斗在农村?】,1,大学生 村官 制度 农村 输血 重要途径 半月谈 记者 河北 山东 陕西 调研 各地 围绕 ...,大学 大学生 学生 村官 困惑 农村 奋斗 农村,大学 大学生 学生 村官 困惑 农村 奋斗 农村 大学生 村官 制度 农村 输血 重要途径 ...
2,从昔日深山的羊肠小道变身今天网友盛赞的“最美赛道”，广西马山县古零镇古零村弄拉屯一条不到4公...,【一个山旮旯的振兴“路”】,1,昔日 深山 羊肠小道 网友 盛赞 最美 赛道 广西 马山县 古镇 古村 屯条 不到 公里 公...,旮旯 振兴,旮旯 振兴 昔日 深山 羊肠小道 网友 盛赞 最美 赛道 广西 马山县 古镇 古村 屯条 不...
3,今年82岁的盖钧镒还时常卷起裤腿下到田间地头。身为中国工程院院士、大豆遗传育种学家，他更是将...,#中国种业十大杰出人物#【盖钧镒：一生为了“豆满仓”】,1,今年 盖钧 时常 卷起 裤腿 田间 地头 身为 中国工程院 院士 大豆 遗传 育种学 更是 ...,中国 种业 十大 杰出 杰出人物 人物 生为 满仓,中国 种业 十大 杰出 杰出人物 人物 生为 满仓 今年 盖钧 时常 卷起 裤腿 田间 地头...
4,"又是一年春播时,广大农民和各级农业部门又开始了繁忙的春耕备耕工作。在乡村振兴政策的引导下,今...",【今年春耕有啥新特点】,1,春播 广大 农民 各级 农业部门 繁忙 春耕 备耕 工作 乡村 振兴 政策 引导 今年 春耕...,今年 春耕 新特 特点,今年 春耕 新特 特点 春播 广大 农民 各级 农业部门 繁忙 春耕 备耕 工作 乡村 振兴...


In [7]:
outputpath='./test_news_with_title_2.csv'
text.to_csv(outputpath,sep=',',index=False,header=True)

In [8]:
a=pd.read_csv('./test_news_with_title_2.csv')
a.head()

,微博正文,标题,分类,微博正文(无标题切词后),标题(切词后),微博正文(有标题切词后)
0,春分已过，清明将至。农谚讲，清明前后，种瓜点豆。当前全国由南向北全面进入春耕生产大忙季节。...,【农业农村部：当前春耕备耕热潮掀起进展顺利】,1,春分 已过 清明 农谚 清明 种瓜 点豆 当前 全国 由南向北 全面 春耕生产 大忙 季节 ...,农业 农村 农村部 当前 春耕 备耕 热潮 掀起 进展 顺利,农业 农村 农村部 当前 春耕 备耕 热潮 掀起 进展 顺利 春分 已过 清明 农谚 清明 ...
1,大学生村官制度是我国向农村输血的重要途径。半月谈记者在河北、山东、陕西等地调研发现，各地围绕...,【大学生村官的困惑：困在农村 还是奋斗在农村?】,1,大学生 村官 制度 农村 输血 重要途径 半月谈 记者 河北 山东 陕西 调研 各地 围绕 ...,大学 大学生 学生 村官 困惑 农村 奋斗 农村,大学 大学生 学生 村官 困惑 农村 奋斗 农村 大学生 村官 制度 农村 输血 重要途径 ...
2,从昔日深山的羊肠小道变身今天网友盛赞的“最美赛道”，广西马山县古零镇古零村弄拉屯一条不到4公...,【一个山旮旯的振兴“路”】,1,昔日 深山 羊肠小道 网友 盛赞 最美 赛道 广西 马山县 古镇 古村 屯条 不到 公里 公...,旮旯 振兴,旮旯 振兴 昔日 深山 羊肠小道 网友 盛赞 最美 赛道 广西 马山县 古镇 古村 屯条 不...
3,今年82岁的盖钧镒还时常卷起裤腿下到田间地头。身为中国工程院院士、大豆遗传育种学家，他更是将...,#中国种业十大杰出人物#【盖钧镒：一生为了“豆满仓”】,1,今年 盖钧 时常 卷起 裤腿 田间 地头 身为 中国工程院 院士 大豆 遗传 育种学 更是 ...,中国 种业 十大 杰出 杰出人物 人物 生为 满仓,中国 种业 十大 杰出 杰出人物 人物 生为 满仓 今年 盖钧 时常 卷起 裤腿 田间 地头...
4,"又是一年春播时,广大农民和各级农业部门又开始了繁忙的春耕备耕工作。在乡村振兴政策的引导下,今...",【今年春耕有啥新特点】,1,春播 广大 农民 各级 农业部门 繁忙 春耕 备耕 工作 乡村 振兴 政策 引导 今年 春耕...,今年 春耕 新特 特点,今年 春耕 新特 特点 春播 广大 农民 各级 农业部门 繁忙 春耕 备耕 工作 乡村 振兴...
